In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import scale, StandardScaler
from sklearn import model_selection
from sklearn.neural_network import MLPRegressor

In [2]:
from warnings import filterwarnings
filterwarnings("ignore")

In [3]:
# Data
df = pd.read_csv("Hitters.csv")
df = df.dropna()
dms = pd.get_dummies(df[['League', 'Division', 'NewLeague']])

y = df["Salary"]
X_ = df.drop(['Salary', 'League', 'Division', 'NewLeague'], axis=1).astype('float64')
X = pd.concat([X_, dms[['League_N', 'Division_W', 'NewLeague_N']]], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.25,
                                                    random_state=42)

### Model & Prediction

In [4]:
scaler = StandardScaler()

In [6]:
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)

In [16]:
scaler.fit(X_test)
X_test_scaled = scaler.transform(X_test)

In [17]:
mlp_model = MLPRegressor().fit(X_train_scaled, y_train)

In [18]:
mlp_model

MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [19]:
mlp_model.predict(X_test_scaled)[0:5]

array([21.31400197, 36.92486371, 71.02832995,  7.77445993, 25.40609892])

In [20]:
y_pred = mlp_model.predict(X_test_scaled)

In [21]:
np.sqrt(mean_squared_error(y_test, y_pred))

657.9157365748823

### Model Tuning

In [23]:
mlp_params = {"alpha": [0.1, 0.01, 0.02, 0.001, 0.0001],
             "hidden_layer_sizes": [(10,20), (5,5), (100,100)]}

In [36]:
mlp_cv_model = GridSearchCV(mlp_model, mlp_params, cv = 10, verbose = 2, n_jobs = -1).fit(X_train_scaled, y_train)

Fitting 10 folds for each of 15 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   15.3s
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:   35.9s finished


In [37]:
mlp_cv_model.best_params_

{'alpha': 0.0001, 'hidden_layer_sizes': (100, 100)}

### Final Model

In [40]:
mlp_tuned = MLPRegressor(alpha = 0.0001, hidden_layer_sizes = (100, 100)).fit(X_train_scaled, y_train)

In [41]:
y_pred = mlp_tuned.predict(X_test_scaled)

In [42]:
np.sqrt(mean_squared_error(y_test, y_pred))

355.1922293384547